迭代器、生成器、装饰器、闭包这几个概念是 Python 中不容易理解透彻的概念。

# 1.迭代器

Python 迭代器（Iterators）对象在遵守迭代器协议时需要支持如下两种方法。

\_\_iter\_\_()，返回迭代器对象自身。这用在 for 和 in 语句中。

\_\_next\_\_()，返回迭代器的下一个值。如果没有下一个值可以返回，那么应该抛出 StopIteration 异常。

```python
class Counter(object):
    def __init__(self, low, high):
        self.current = low
        self.high = high

    def __iter__(self):
        return self

    def __next__(self):
        #返回下一个值直到当前值大于 high
        if self.current > self.high:
            raise StopIteration
        else:
            self.current += 1
            return self.current - 1
```
现在我们能把这个迭代器用在我们的代码里。
```python
c = Counter(5,10)
for i in c:
    print(i, end=' ')

# 5 6 7 8 9 10
```

请记住迭代器只能被使用一次。这意味着迭代器一旦抛出 StopIteration，它会持续抛出相同的异常。

我们已经看过在 for 循环中使用迭代器的例子了，下面的例子试图展示迭代器被隐藏的细节：
```python
iterator = iter(c)
while True:
    try:
        x = iterator.__next__()
        print(x, end=' ')
    except StopIteration as e:
        break

# 5 6 7 8 9 10
```


# 2.生成器

生成器（Generators）:生成器是更简单的创建迭代器的方法，这通过在函数中使用 yield 关键字完成：
```python
def my_generator():
    print("Inside my generator")
    yield 'a'
    yield 'b'
    yield 'c'

my_generator()
# <generator object my_generator at 0x7fbcfa0a6aa0
```

在上面的例子中我们使用 yield 语句创建了一个简单的生成器。我们能在 for 循环中使用它，就像我们使用任何其它迭代器一样。

```python
for char in my_generator():
    print(char)
# Inside my generator
# a
# b
# c
```

在下一个例子里，我们会使用一个生成器函数完成与 Counter 类相同的功能，并且把它用在 for 循环中。

```python
def counter_generator(low, high):
    while low <= high:
        yield low
        low += 1

for i in counter_generator(5,10):
    print(i, end=' ')
 
# 5 6 7 8 9 10
```

在 While 循环中，每当执行到 yield 语句时，返回变量 low 的值并且生成器状态转为挂起。在下一次调用生成器时，生成器从之前冻结的地方恢复执行然后变量 low 的值增一。生成器继续 while 循环并且再次来到 yield 语句...

当你调用生成器函数时它返回一个生成器对象。如果你把这个对象传入 dir() 函数，你会在返回的结果中找到 \_\_iter\_\_ 和 \_\_next\_\_ 两个方法名
```python
c = counter_generator(5, 10)
dir(c)
```

我们通常使用生成器进行惰性求值。这样使用生成器是处理大数据的好方法。如果你不想在内存中加载所有数据，你可以使用生成器，一次只传递给你一部分数据。

os.path.walk() 函数是最典型的这样的例子，它使用一个回调函数和当前的 os.walk 生成器。使用生成器实现节约内存。

我们可以使用生成器产生无限多的值。以下是一个这样的例子。

```python
def infinite_generator(start=0):
    while True:
        yield start
        start += 1

for num in infinite_generator(4):
    print(num, end=' ')
    if num > 20:
        break

# 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21
```

如果我们回到 my_generator() 这个例子，我们会发现生成器的一个特点：**它们是不可重复使用的**.

一个创建可重复使用生成器的方式是不保存任何状态的基于对象的生成器。任何一个生成数据的含有 \_\_iter\_\_ 方法的类都可以用作对象生成器。在下面的例子中我们重新创建了 counter 生成器。

```python
class Counter(object):
    def __init__(self, low, high):
        self.low = low
        self.high = high
        
    def __iter__(self):
        counter = self.low
        while self.high >= counter:
            yield counter
            counter += 1

gobj = Counter(5, 10)
for num in gobj:
    print(num, end=' ')

# 5 6 7 8 9 10
for num in gobj:
    print(num, end=' ')

# 5 6 7 8 9 10
```

上面的 gobj 并不是生成器或迭代器，因为它不具有 \_\_next\_\_ 方法，只是一个可迭代对象，生成器是一定不能重复循环的。 如果想要使类的实例变成迭代器，可以用 \_\_iter\_\_ + \_\_next\_\_ 方法实现：

```python
from collections import Iterator
class Test():
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def __iter__(self):
        return self
    def __next__(self):
        self.a += 1
        if self.a > self.b:
            raise StopIteration()
        return self.a    

test = Test(5, 10)
isinstance(test, Iterator)
# True
```

### 生成器表达式

生成器表达式（Generator expressions）:生成器表达式是列表推导式和生成器的一个高性能，内存使用效率高的推广。
举个例子，我们尝试对 1 到 9 的所有数字进行平方求和。
```python
sum([x*x for x in range(1,10)])
```
这个例子实际上首先在内存中创建了一个平方数值的列表，然后遍历这个列表，最终求和后释放内存。你能理解一个大列表的内存占用情况是怎样的。
我们可以通过使用生成器表达式来节省内存使用。
```python
sum(x*x for x in range(1,10))
```
生成器表达式的语法要求其总是直接在在一对括号内，并且不能在两边有逗号。这基本上意味着下面这些例子都是有效的生成器表达式用法示例：
```python
sum(x*x for x in range(1,10))
# 285
g = (x*x for x in range(1,10))
g
# <generator object <genexpr> at 0x7fc559516b90>
```

# 3.装饰器

装饰器（Decorators）用来给一些对象动态的添加一些新的行为，我们使用过的闭包也是这样的。

我们会创建一个简单的示例，将在函数执行前后打印一些语句。
```python
def my_decorator(func):
     def wrapper(*args, **kwargs):
        print("Before call")
        result = func(*args, **kwargs)
        print("After call")
        return result
     return wrapper

@my_decorator
def add(a, b):
     #我们的求和函数
    return a + b

add(1, 3)
# Before call
# After call
# 4
```

# 4.闭包

闭包（Closures）是由另外一个函数返回的函数。我们使用闭包去除重复代码。在下面的例子中我们创建了一个简单的闭包来对数字求和。
```python
def add_number(num):
    #adder 是一个闭包
    def adder(number):
        return num + number
    
    return adder

a_10 = add_number(10)
a_10(21)
# 31
a_10(34)
# 44
a_5 = add_number(5)
a_5(3)
# 8
```
adder 是一个闭包，把一个给定的数字与预定义的一个数字相加。

In [1]:
def counter_generator(low, high):
    while low <= high:
        yield low
        low += 1
        
c = counter_generator(5, 10)
dir(c)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

# Virtualenv使用

使用pip3安装virtualenv
```
sudo pip3 install virtualenv
```

我们会创建一个叫做 virtual 的目录，在里面我们会有两个不同的虚拟环境。

```
mkdir virtual
```

下面的命令创建一个叫做 virt1 的环境。
```
cd virtual
virtualenv virt1
```

现在我们激活这个 virt1 环境。
```
ource virt1/bin/activate
# (virt1)shiyanlou：~/
```
提示符的第一部分是当前虚拟环境的名字，当你有多个环境的时候它会帮助你识别你在哪个环境里面。
现在我们将安装 redis 这个 Python 模块。
```
(virt1) sudo pip3 install redis
```
使用 deactivate 命令关闭虚拟环境。
```
(virt1) deactivate
```
现在我们将创建另一个虚拟环境 virt2，我们会在里面同样安装 redis 模块，但版本是 2.8 的旧版本
```
 virtualenv virt2
 source virt2/bin/activate
(virt2) sudo pip3 install redis==2.8
```
这样可以为你的所有开发需求拥有许多不同的环境。

# 项目包管理

这里阐述了一个完整的 Python 项目结构，你可以使用什么样的目录布局以及怎样发布软件到网络上。

知识点

* 创建项目，编写 __init__ 文件
* 使用 setuptools 模块，编写 setup.py 和 MANIFEST.in 文件
* 创建源文件的发布版本
* 项目注册&上传到 PyPI

## 1.创建python项目目录

我们的项目名为 factorial，放到 /home/my/factorial 目录
我们给将要创建的 Python 模块取名为 myfact，因此我们下一步创建 myfact 目录。
```python
# linux
mkdir myfact
cd myfact/
```

主代码将在 fact.py 文件里面。

```python

"myfact module"

def factorial(num):
    """
    返回给定数字的阶乘值

    :arg num: 我们将计算其阶乘的整数值

    :return: 阶乘值，若传递的参数为负数，则为 -1
    """
    if num >= 0:
        if num == 0:
            return 1
        return num * factorial(num -1)
    else:
        return -1
```
我们还有模块的 \_\_init\_\_.py 文件，内容如下：
```python
from fact import factorial
__all__ = [factorial, ]
```

我们还在 factorial 目录下添加了一个 README.rst 文件。因此，目录结构看起来像下面这样：
```
factorial
|--myfact
|  |--fact.py
|  |--__init__.py
|--readme.rst
|--MAINFEST.in
```

现在我们要写一个 factorial/MANIFEST.in 文件，它用来在使用 sdist 命令的时候找出将成为项目源代码压缩包一部分的所有文件。
```
include *.py
include README.rst
```

如果你想要排除某些文件，你可以在这个文件中使用 exclude 语句。

最终我们需要写一个 factorial/setup.py，用来创建源代码压缩包或安装软件。
安装python-setuptools 包
```
sudo pip3 install setuptools
```
编写setup.py文件
```python
"""Factorial project"""
from setuptools import find_packages, setup

setup(name = 'factorial',
    version = '0.1',
    description = "Factorial module.",
    long_description = "A test module for our book.",
    platforms = ["Linux"],
    author="ShiYanLou",
    author_email="support@mcy.com",
    url="https://www.mcy.com",
    license = "MIT",
    packages=find_packages()
    )

```
name 是项目名称，version 是发布版本，description 和 long_description 分别是项目介绍，项目长描述。platforms 是此模块的支持平台列表。find_packages() 是一个能在你源目录下找到所有模块的特殊函数，packaging docs。

## 2.创建源码安装包

要创建一个源文件发布版本，执行以下命令。
```
python3 setup.py sdist
```
执行完返回如下信息：
```
running sdist
running egg_info
creating factorial.egg-info
writing factorial.egg-info/PKG-INFO
writing top-level names to factorial.egg-info/top_level.txt
writing dependency_links to factorial.egg-info/dependency_links.txt
writing manifest file 'factorial.egg-info/SOURCES.txt'
reading manifest file 'factorial.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'factorial.egg-info/SOURCES.txt'
running check
creating factorial-0.1
creating factorial-0.1/factorial.egg-info
creating factorial-0.1/myfact
making hard links in factorial-0.1...
hard linking MANIFEST.in -> factorial-0.1
hard linking README.rst -> factorial-0.1
hard linking setup.py -> factorial-0.1
hard linking factorial.egg-info/PKG-INFO -> factorial-0.1/factorial.egg-info
hard linking factorial.egg-info/SOURCES.txt -> factorial-0.1/factorial.egg-info
hard linking factorial.egg-info/dependency_links.txt -> factorial-0.1/factorial.egg-info
hard linking factorial.egg-info/top_level.txt -> factorial-0.1/factorial.egg-info
hard linking myfact/__init__.py -> factorial-0.1/myfact
hard linking myfact/fact.py -> factorial-0.1/myfact
Writing factorial-0.1/setup.cfg
creating dist
Creating tar archive
removing 'factorial-0.1' (and everything under it)
```
我们能在 dist 目录下看到一个 tar 压缩包。

```
ls dist/
# factorial-0.1.tar.gz
```

执行下面的命令从源代码安装。

```
sudo python3 setup.py install
```
学习更多可前往 packaging.python.org。

## 3.包管理

你还记得我们经常使用的 pip 命令吗？有没有想过这些包是从哪里来的？答案是 PyPI。这是 Python 的软件包管理系统。
为了实验，我们会使用 PyPI 的测试服务器 https://testpypi.python.org/pypi。

### 3.1 创建账号

首先在这个链接https://test.pypi.org/ 注册账号。你会收到带有链接的邮件，点击这个链接确认你的注册。

创建 ~/.pypirc 文件，存放你的账号详细信息，其内容格式如下：
```
[distutils]
index-servers = pypi
    testpypi

[pypi]
repository: https://upload.pypi.org/legacy/
username: <username>
password: <password>

[testpypi]
repository:https://test.pypi.org/legacy/
username: <username>
password: <password>
```

替换 <username> 和 <password> 为您新创建的帐户的详细信息。在这里，由于我们是到 testpypi的网页上去注册账号，即将相应的服务上传到 testpypi，所以在这里，你只需修改[testpypi]的用户名和密码

记得在 setup.py 中更改项目的名称为其它的名字来测试下面的指令，在接下来的命令中我将项目名称修改为factorial2，为了不重复，需要自行修改至其它名称。

### 3.2 上传到 TestPyPI 服务

下一步我们会将我们的项目到 TestPyPI 服务。这通过 twine 命令完成。

我们也会使用 -r 把它指向测试服务器。
```
sudo pip3 install twine
wine upload dist/* -r testpypi
```
执行完毕会返回类似下面的信息,
```
Uploading distributions to https://test.pypi.org/legacy/
Uploading factorial2-0.1.tar.gz
```
在这里你也可以使用下面的命令上传到 PyPI 服务上，但这里需要注意，在 ~/.pypirc 里面，你需要到 https://pypi.python.org页面，按照上面的步骤去注册一个账号，然后到~/.pypirc 的 [pypi] 下填写相应的用户名和密码。testpypi 和 pypi 的账号密码并不通用。
```
twine upload dist/* -r testpypi
```
现在如果你浏览这个页面，你会发现你的项目已经准备好被别人使用了。